<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NB10_99__3DP_3_Data%20Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Melhorias da sessão**
* Desenvolver a sessão sobe WOE.

___
# **Referências**
* [Why, How and When to Scale your Features](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e)
* [Demonstrating the different strategies of KBinsDiscretizer](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html#sphx-glr-auto-examples-preprocessing-plot-discretization-strategies-py);
* [Why do we need feature scaling in Machine Learning and how to do it using SciKit Learn?](https://medium.com/@contactsunny/why-do-we-need-feature-scaling-in-machine-learning-and-how-to-do-it-using-scikit-learn-d8314206fe73)
* [Importance of Feature Scaling](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py) --> Muito importante por demonstrar os efeitos e a importância de se transformar as colunas numéricas.
* [Feature discretization](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_classification.html#sphx-glr-auto-examples-preprocessing-plot-discretization-classification-py) --> Mostra o impacto na acurácia dos modelos com e sem discretização. Ou seja, discretizar faz sentido!

___
# **Agenda**

___
# **3DP_DATA TRANSFORMATION**

___
# **Machine Learning com Python (Scikit-Learn)**

![Scikit-Learn](https://github.com/MathMachado/Materials/blob/master/scikit-learn-1.png?raw=true)

## Carregar as bibliotecas (genéricas) Python

In [0]:
!pip install category_encoders
!pip install update
!pip install bamboolib

In [0]:
import pandas as pd
#from pandas import Series, DataFrame

import numpy as np
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')

import category_encoders as ce # library para aplicação do WOE - Weight Of Evidence para avaliar importância dos atributos

# remove warnings to keep notebook clean
import warnings
warnings.filterwarnings('ignore')

## Carregar os dados

### Dataframe gerado aleatoriamente - variáveis com distribuição Normal

In [0]:
N= 1000
df_A1 = pd.DataFrame({
    'coluna1': np.random.normal(0, 2, N),
    'coluna2': np.random.normal(50, 3, N),
    'coluna3': np.random.normal(-5, 5, N),
    'coluna4': np.random.normal(-10, 10, N)
})
df_A1.head()

In [0]:
N= 1000
df_A1 = pd.DataFrame({
    'coluna1': np.random.normal(0, 2, N),
    'coluna2': np.random.normal(50, 3, N),
    'coluna3': np.random.exponential(1, N),
    'coluna4': np.random.normal(-10, 10, N)
})
df_A1.head()

In [0]:
df_A1.shape

### Dataframe gerado aleatoriamente 2

In [0]:
from sklearn.datasets import make_classification

classification_data, classification_class = make_classification(n_samples=N, n_features=4, n_informative=3, n_redundant=1, n_classes=3)

df_A2 = pd.DataFrame({'coluna1':classification_data[:,0],
                                  'coluna2':classification_data[:,1],
                                  'coluna3':classification_data[:,2],
                                  'coluna4':classification_data[:,3],
                                  'coluna5':classification_class})
df_A2.head()

In [0]:
df_A2.shape

___
# **Transformações**

## StandardScaler
* Assume que os dados (as colunas a serem transformadas) são normalmente distribuidos ;
* Se os dados não possuem distribuição Normal, então esta não é uma boa transformação a se aplicar.

$$z_{i}= \frac{x_{i}-mean(x)}{std(x)}$$

### Exemplo

In [0]:
df_A1.head()

Histograma:

In [0]:
plt.hist(df_A1['coluna3'], color = 'blue', edgecolor = 'black', bins = int(180/5))

# Adiciona títulos e labels
plt.title('Histograma da Variável3')

Considere o gráfico a seguir:

In [0]:
df_A1.plot.kde()

Qual a interpretação para esse gráfico?

A seguir, vamos aplicar a transformação StandardScaler as variáveis

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
df_A1_StandardScaler = StandardScaler().fit_transform(df_A1)
df_A1_StandardScaler = pd.DataFrame(df_A1_StandardScaler, columns=['Coluna1','Coluna2','Coluna3', 'Coluna4'])

Agora compare esse novo gráfico abaixo

In [0]:
df_A1_StandardScaler.plot.kde()

Qual a conclusão?

## MinMaxScaler
* **Transformação muito popular e utilizada**.
* Transforma os dados para o intervalo (0, 1);
* Se StandardScaler não é aplicável, então essa transformação funciona bem.
* Sensível aos outliers. Portanto, o ideal é que os outliers sejam tratados previamente.

$$z_{i}= \frac{x_{i}-min(x)}{max(x)-min(x)}$$

### Exemplo

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
df_A1.plot.kde()

In [0]:
#minmax = MinMaxScaler()
df_A1_MinMaxScaler = MinMaxScaler().fit_transform(df_A1)
df_A1_MinMaxScaler = pd.DataFrame(df_A1_MinMaxScaler,columns=['Coluna1','Coluna2','Coluna3', 'Coluna4'])
df_A1_MinMaxScaler.plot.kde()

Qual a conclusão?

## RobustScaler
* Transformação ideal para dados com outliers.

$$z_{i}= \frac{x_{i}-Q_{1}(x)}{Q_{3}(x)-Q_{1}(x)}$$

In [0]:
df_A1.plot.kde()

In [0]:
from sklearn.preprocessing import RobustScaler

In [0]:
df_A1_RobustScaler = RobustScaler().fit_transform(df_A1)
df_A1_RobustScaler = pd.DataFrame(df_A1_RobustScaler, columns=['Coluna1','Coluna2','Coluna3', 'Coluna4'])
df_A1_RobustScaler.plot.kde()

## Encoding Variáveis Categoricas

### Encoding Variáveis Ordinais
* Exemplo: Variáveis com valores ordinais: Low, Medium ou High.

#### Gera um dataframe como exemplo.

In [0]:
# Aqui vou usar a função randint - Retorna números inteiros aleatórios incluindo o número inferior e excluindo o superior.

l_Idade= [np.random.randint(20, 40), np.random.randint(20, 40), np.random.randint(20, 40), np.random.randint(20, 40), np.random.randint(20, 40),
         np.random.randint(20, 40), np.random.randint(20, 40),np.random.randint(20, 40),np.random.randint(20, 40),np.random.randint(20, 40)]

l_Salario= ['Low', 'Medium', 'High']
l_Salario2= np.random.choice(l_Salario, 10, p=[0.6, 0.3, 0.1])

df_A3 = pd.DataFrame({
    'Idade': l_Idade,
    'Salario': l_Salario2})

In [0]:
df_A3

Neste exemplo, vamos redefinir a variável categórical ordinal 'Salario' da seguinte forma:

In [0]:
df_A3['Salario_Cat']= df_A3.Salario.map({'Low':1,'Medium':2,'High':3})
df_A3

### Encoding Variáveis Nominais
* Exemplo: Variáveis com valores nominais: Sexo (Feminino, masculino).

* Use One-Hot Encoding ou pd.get.dummies()

Vamos utilizar o dataframe criado no passo anterior:

In [0]:
df_A3['Salario'].unique()

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#### Aplicar LabelEncoder()

In [0]:
le = LabelEncoder()
df_A3['Salario_le'] = le.fit_transform(df_A3['Salario'])
df_A3

#### Aplicar pd.get.dummies()

In [0]:
dummies= pd.get_dummies(df_A3['Salario'])
df_A3= pd.concat([df_A3, dummies], axis= 1)
df_A3

O comando a seguir produz o mesmo resultado que o anterior:
```
df_A3= df_A3.merge(dummies, how= 'left')
```

___
# **Exercícios**
> Para cada um dos dataframes a seguir, aplique os seguintes steps:

* Padronizar o nome das colunas
    * Eliminar espaços entre os nomes das colunas;
    * Eliminar caracteres especiais dos nomes das colunas;
    * Renomear as colunas com lower() (ou upper());
* Aplicar a trasformação StandardScaler e MinMaxScaler em cada uma das colunas do dataframe;
* DataViz - Mostrar a distribuição das colunas antes e depois das transformações num mesmo gráfico para que sejamos capazes de comparar o comportamento das colunas;


## Exercício 1 - Breast Cancer

In [0]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X= cancer['data']
y= cancer['target']

df_A1_cancer = pd.DataFrame(np.c_[X, y], columns= np.append(cancer['feature_names'], ['target']))
df_A1_cancer['target'] = df_A1_cancer['target'].map({0: 'malign', 1: 'benign'})
df_A1_cancer.head()

## Exercício 2 - Boston Housing Price

In [0]:
from sklearn.datasets import load_boston

boston = load_boston()
X= boston['data']
y= boston['target']

df_A1_boston = pd.DataFrame(np.c_[X, y], columns= np.append(boston['feature_names'], ['target']))
df_A1_boston.head()

## Exercício 3 - Iris
* [Aqui](https://en.wikipedia.org/wiki/Iris_flower_data_set) você obterá mais informações sobre o dataframe iris. Confira.

In [0]:
from sklearn.datasets import load_iris

iris = load_iris()
X= iris['data']
y= iris['target']

df_iris = pd.DataFrame(np.c_[X, y], columns= np.append(iris['feature_names'], ['target']))
df_iris['target2'] = df_iris['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
df_iris.head()

In [0]:
df_iris.columns = [c.replace(' ', '_') for c in df_iris.columns]
df_iris.columns = [c.replace('_(cm)', '') for c in df_iris.columns]
df_iris.head()

In [0]:
df_iris.plot.kde()

In [0]:
df_iris_MinMaxScaler = MinMaxScaler().fit_transform(df_iris[['sepal_length','sepal_width','petal_length', 'petal_width']])
df_iris_MinMaxScaler = pd.DataFrame(df_iris_MinMaxScaler, columns=['sepal_length','sepal_width','petal_length', 'petal_width'])
df_iris_MinMaxScaler.plot.kde()

## Exercícios 4 - Diabetes

In [0]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X= diabetes['data']
y= diabetes['target']

df_A1_diabetes = pd.DataFrame(np.c_[X, y], columns= np.append(diabetes['feature_names'], ['target']))
df_A1_diabetes.head()

# WOE - Weight Of Evidence
* As vantagens da transformação WOE são
    * Lida bem com NaN's;
    * Lida bem com outliers;
    * A transformação é baseada no valor logarítmico das distribuições.
    * Usando a técnica de binning apropriada, pode estabelecer uma relação monotônica (aumentar ou diminuir) entre a variável dependente e independente.